Dans ce TP, nous allons implémenter une collecte simple de données via une API : https://rapidapi.com/edamam/api/edamam-food-and-grocery-database. Cela permet d'enrichir des jeux de données d'images lorsque notre problématique requiert par exemple des images spécifiques.

Le but est d'obtenir une extraction des produits à base de champagne dans un fichier “.csv”, contenant pour chaque produit les données suivantes : foodId, label, category, foodContentsLabel, image, ainsi que les images correspondantes.

Rassembler les imports dans la cellule suivante.

In [11]:
import os

from dotenv import load_dotenv
import pandas as pd
import requests

La communication avec une API implique de s'identifier. Cela se fait par l'intermédiaire d'une clef. C'est une bonne pratique de ne pas laisser tout ce qui touche à l'identification en dur dans le code si celui-ci est amené à être partagé. Il existe plusieurs manières d'éviter cet écueil, en voici un bon résumé : https://blog.gitguardian.com/how-to-handle-secrets-in-python/.

Ici, pour accélérer la mise en place et ne pas vous faire créer un compte, je vous partage une clef API d'un compte test. Pour autant, nous suivrons le principe de séparer la clef du code en utilisant la méthode 1 du résumé, à savoir par l'intermédiaire d'un fichier .env. ainsi que la librairie dotenv (nom du module), qui nécessitera une installation (nom du package : python-dotenv).



Récupérer les variables d'environnement du fichier .env en appelant load_dotenv(). Utiliser os.getenv pour stocker la valeur de la clef API dans une variable. Vérifier TEMPORAIREMENT que la clef a bien été chargée en l'affichant.

In [12]:
load_dotenv()
API_KEY = os.getenv("API_KEY")

En observant cette requête exemple :

![Code Snippet](../../../docs/images/rapidapi.png)

Préparer les champs d'une requête permettant de récolter les produits appartenant au type de nutrition "cooking" et contenant au moins l'ingrédient "champagne". <br>
Trouver le nommage exact des paramètres à renseigner dans la requête grâce à la documentation des paramètres optionnels de l'API.

In [13]:
url = "https://edamam-food-and-grocery-database.p.rapidapi.com/api/food-database/v2/parser"

headers = {
    "x-rapidapi-key": API_KEY,
    "x-rapidapi-host": "edamam-food-and-grocery-database.p.rapidapi.com",
}

params = {
    "nutrition-type": "cooking",
    "ingr": "champagne",
}

A l'aide de la bibliothèque requests, envoyer la requête GET correspondante à l'API. Stocker l'output dans une variable et en faire un affichage.

In [14]:
response = requests.get(url, params=params, headers=headers)

La fonction print renvoie le code de la réponse, permettant de confirmer qu'elle fut un succès ou qu'elle a rencontré une erreur. Accéder désormais au contenu de la réponse en utilisant la méthode .json().

In [15]:
print(response)

<Response [200]>


In [16]:
response.json()

{'text': 'champagne',
 'count': 298,
 'parsed': [{'food': {'foodId': 'food_a656mk2a5dmqb2adiamu6beihduu',
    'uri': 'http://www.edamam.com/ontologies/edamam.owl#Food_table_white_wine',
    'label': 'Champagne',
    'knownAs': 'dry white wine',
    'nutrients': {'ENERC_KCAL': 82.0,
     'PROCNT': 0.07,
     'FAT': 0.0,
     'CHOCDF': 2.6,
     'FIBTG': 0.0},
    'category': 'Generic foods',
    'categoryLabel': 'food',
    'image': 'https://www.edamam.com/food-img/a71/a718cf3c52add522128929f1f324d2ab.jpg'}}],
 'hints': [{'food': {'foodId': 'food_a656mk2a5dmqb2adiamu6beihduu',
    'uri': 'http://www.edamam.com/ontologies/edamam.owl#Food_table_white_wine',
    'label': 'Champagne',
    'knownAs': 'dry white wine',
    'nutrients': {'ENERC_KCAL': 82.0,
     'PROCNT': 0.07,
     'FAT': 0.0,
     'CHOCDF': 2.6,
     'FIBTG': 0.0},
    'category': 'Generic foods',
    'categoryLabel': 'food',
    'image': 'https://www.edamam.com/food-img/a71/a718cf3c52add522128929f1f324d2ab.jpg'},
   'measur

Reconstruire le dataframe correspondant aux données qui nous intéressent (pour rappel, seuls les champs suivants nous intéressent : foodId, label, category, foodContentsLabel, image).

In [17]:
wanted_cols = ["foodId", "label", "category", "foodContentsLabel", "image"]

In [18]:
filtered_champagne = [
    {
        col: hint["food"].get(col)
        for col in wanted_cols
    }
    for hint in response.json()["hints"]
]

filtered_champagne_df = pd.DataFrame(data=filtered_champagne)

In [19]:
data = [hint["food"] for hint in response.json()["hints"]]

filtered_champagne_df = pd.DataFrame(
    data=data,
    columns=wanted_cols,
)

Afficher les 10 premiers éléments du dataframe ainsi que sa taille totale.

In [20]:
filtered_champagne_df.head(10)

,foodId,label,category,foodContentsLabel,image
0,food_a656mk2a5dmqb2adiamu6beihduu,Champagne,Generic foods,NaN,https://www.edamam.com/food-img/a71/a718cf3c52...
1,food_b8d1c2rbdpdok6bjpwmtlbsa5hs4,"Champagne Vinaigrette, Champagne",Packaged foods,OLIVE OIL; BALSAMIC VINEGAR; CHAMPAGNE VINEGAR...,NaN
2,food_axmullnbxsm8f5aug13l2aqoyu6j,"Champagne Vinaigrette, Champagne",Packaged foods,INGREDIENTS: WATER; CANOLA OIL; CHAMPAGNE VINE...,https://www.edamam.com/food-img/d88/d88b64d973...
3,food_a7j75b0bk4sd87a34ed3tats8n31,"Champagne Dressing, Champagne",Packaged foods,OLIVE OIL; BALSAMIC VINEGAR; CHAMPAGNE VINEGAR...,NaN
4,food_b2xjyr5beqff2mbwycgjza8o1hxk,"Champagne Vinaigrette, Champagne",Packaged foods,CANOLA AND SOYBEAN OIL; WHITE WINE (CONTAINS S...,NaN
5,food_a2z6lg7a3qtqbbbsw4yxgbx8ql4q,"Champagne Dressing, Champagne",Packaged foods,SOYBEAN OIL; WHITE WINE (PRESERVED WITH SULFIT...,https://www.edamam.com/food-img/ab2/ab2459fc2a...
6,food_a7d4hjabk0x5sqby8om6gbf5c4tr,"Champagne Truffles, Champagne",Packaged foods,NON-GMO MILK CHOCOLATE (CANE SUGAR; FULL CREAM...,NaN
7,food_alpl44taoyv11ra0lic1qa8xculi,Champagne Buttercream,Generic meals,sugar; butter; shortening; vanilla; champagne;...,NaN
8,food_agum7rba8gepbgajiid4zbgmcwad,"Champagne Punch Concentrate, Champagne",Packaged foods,HIGH FRUCTOSE CORN SYRUP; WATER; LESS THAN 2%:...,NaN
9,food_arebcyiaufsqpnax8zj38bipaerf,"Champagne Cotton Candy, Champagne",Packaged foods,SUGAR; ARTIFICIAL & NATURAL FLAVOR.,NaN


Pour certaines lignes, l'image n'est pas disponible et dans ce cas l'URL est à None. Afficher le nombre de lignes dont l'image dispose d'une URL.

In [21]:
filtered_champagne_df["foodId"].count()

20

Enregistrer cette extraction au format csv dans le dossier courant.

In [22]:
filtered_champagne_df.to_csv("champagne_extract.csv")

Finalement, récupérer les images dont l'URL est renseignée, à l'aide de .content. Les sauvegarder dans le dossier "images". Vérifier que le nombre de fichiers téléchargés correspond bien au nombre d'images attendu.

In [ ]:
url = "https://www.edamam.com/food-img/a71/a718cf3c52add522128929f1f324d2ab.jpg"

response = requests.get(url="https://www.edamam.com/food-img/a71/a718cf3c52add522128929f1f324d2ab.jpg")

In [ ]:
response.content

b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xfe\x00;CREATOR: gd-jpeg v1.0 (using IJG JPEG v62), quality = 80\n\xff\xdb\x00C\x00\x06\x04\x05\x06\x05\x04\x06\x06\x05\x06\x07\x07\x06\x08\n\x10\n\n\t\t\n\x14\x0e\x0f\x0c\x10\x17\x14\x18\x18\x17\x14\x16\x16\x1a\x1d%\x1f\x1a\x1b#\x1c\x16\x16 , #&\')*)\x19\x1f-0-(0%()(\xff\xdb\x00C\x01\x07\x07\x07\n\x08\n\x13\n\n\x13(\x1a\x16\x1a((((((((((((((((((((((((((((((((((((((((((((((((((\xff\xc0\x00\x11\x08\x01,\x01,\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1c\x00\x01\x00\x02\x03\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x06\x04\x05\x07\x02\x03\x08\xff\xc4\x00L\x10\x00\x02\x01\x03\x02\x03\x03\x05\t\x0b\n\x07\x01\x00\x00\x00\x00\x01\x02\x03\x04\x11\x05\x06\x12!1\x13AQ\x07"aq\x81\x142Rr\x91\xa1\xb1\xc1\xe2\x15\x16#$4BEU\x93\x94\xb235Cbes\x83\x84\xa2\xd1\x17%Scd\x82\x92\xa3\xff\xc4\x00\x1a\x01\x01\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x01\x03\x04\x05\x06\xff\xc4\x00 \x11\x01\x01

In [ ]:
num = 0
for url in filtered_champagne_df["image"]:
    if not pd.isnull(url):
        img_response = requests.get(url=url).content
        with open(
            os.path.join("images", f"champagne_{num}.jpg"),
            "wb",
        ) as file:
            file.write(img_response)
        num += 1